In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
from bs4 import BeautifulSoup
import pytz
from datetime import datetime, timezone, timedelta, date
import gspread
from google.oauth2.service_account import Credentials

In [2]:
os.environ["GOOGLE_VALOREO_SOURCING_CREDENTIALS"]="../consol_inventario/valoreo_sourcing.json"
path_to_credentials_file = os.environ.get('GOOGLE_VALOREO_SOURCING_CREDENTIALS')
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(path_to_credentials_file, scopes=scope)

In [3]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "./downloads"}
chromeOptions.add_experimental_option("prefs",prefs)

In [28]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chromeOptions)

In [29]:
driver.maximize_window()
driver.get("https://sgl.homecenter.co/#/auth/login")

In [30]:
user = '901486566'
psw = 'aM4b5Ia*'

In [7]:
driver.find_element(By.XPATH, '/html/body/app/login-2/div/div[2]/div[3]/form/mat-form-field[1]/div/div[1]/div[3]/input').send_keys(user)
driver.find_element(By.XPATH, '/html/body/app/login-2/div/div[2]/div[3]/form/mat-form-field[2]/div/div[1]/div[3]/input').send_keys(psw)
driver.find_element(By.XPATH, '/html/body/app/login-2/div/div[2]/div[3]/form/button').click()
time.sleep(40)
try:
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
except:
    pass
try:
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/mat-dialog-container/app-notificacion/div[2]/div[2]/button').click()
except:
    pass
    

a = driver.find_elements(By.TAG_NAME, 'mat-card')
for card in a[1:]:
    if 'Informacion Logistica' in card.text:
        card.click()
        break
time.sleep(60)
frame = driver.find_element(By.XPATH, '/html/body/app/vertical-layout-1/div/div/div/div/content/app-iframe/div/iframe')
driver.switch_to.frame(frame)
time.sleep(60)
driver.find_element(By.XPATH, '/html/body/app/div/div/div/div/div[2]/div[2]/div/div/div/div/div/div/app-home/form/main/section/div[1]/mat-card[2]/mat-card-content/button').click()
time.sleep(40)
driver.quit()

In [24]:
files = os.listdir('../../../../Downloads')
for file in files:
    if '.xlsx' in file:
        selected_file = file
homecenter_inv = pd.read_excel(f'../../../../Downloads/{selected_file}', sheet_name='Reporte Inventario')
os.remove(f'../../../../Downloads/{selected_file}')

In [40]:
df_catalog = pd.read_csv('sku-ean.csv', dtype=str)

In [41]:
homecenter_inv['Sku'] = homecenter_inv['Sku'].astype(str)

In [42]:
homecenter_inv = homecenter_inv.merge(df_catalog, how='left', left_on='Sku', right_on='SKU').drop(columns='SKU')

In [51]:
local_tz = pytz.timezone('America/Mexico_City')
current_day = datetime.today().astimezone(local_tz).strftime('%Y%m%d')
homecenter_inv.to_csv(f'homecenter_inv.csv', index=False)
homecenter_inv.to_csv(f'{current_day}_homecenter_inv.csv', index=False)